In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import io
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#Lecture des dataset
features = pd.read_csv('/content/features.csv')
coded_ids_labels_train = pd.read_csv('/content/coded_ids_labels_train.csv')
coded_ids = pd.read_csv("/content/coded_ids.csv")
coded_ids_labels_test = pd.read_csv("/content/coded_ids_labels_test.csv")

#Merge des dataset
ids_train_merge=pd.merge(coded_ids,coded_ids_labels_train,how="right")
features_merge=pd.merge(ids_train_merge, features, how="left")


features_merge.drop(['utc_offset', 'urls_used_on_average', 'followers_count_minus_2002', 'time_zone' ,'lang', 'date_newest_tweet', 'date_oldest_tweet', 'max_intertweet_times', 'min_intertweet_times', 'std_intertweet_times', 'avg_intertweet_times'], inplace=True, axis=1)


df = features_merge.dropna(axis=1, how='all')

df = df._get_numeric_data()
df[df < 0] = 0

features_merge['default_profile']=features_merge['default_profile'].replace({True :1, False: 0})
features_merge['default_profile_image']=features_merge['default_profile_image'].replace({True :1, False: 0})


df.describe()

df.corr()

X = df.iloc[:,1:]
X

X.isnull().sum()

Y = df.iloc[:,2].values
Y

train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size = 0.2)

mnb = MultinomialNB(alpha=1.9)         

mnb.fit(train_x,train_y)
y_pred1 = mnb.predict(test_x)
print("Score de précision pour Naive Bayes : ", accuracy_score(y_pred1,test_y))
print ("Le rappel pour Naive Bayes : ", recall_score(test_y,y_pred1))

print(y_pred1)



#remplissage du fichier de prediction 

test_id=pd.merge(coded_ids,coded_ids_labels_test,on="coded_id",how="right") #goal get the features
test_id

test=pd.merge(test_id,features,on="user_id",how="left")
test

test=test.drop(list(test.select_dtypes(object).columns),axis=1)
test["utc_offset"].fillna(test["utc_offset"].mean(),inplace=True)
del test["spam_in_screen_name"]


test['default_profile']=test['default_profile'].replace({True :1, False: 0})
test['default_profile_image']=test['default_profile_image'].replace({True :1, False: 0})


first_col = test.pop("label")
test.insert(0, "label", first_col)

print(test)
test["label"]=mnb.predict(test.iloc[:,2:136])
print(test)

res=pd.merge(test[["coded_id","label"]],coded_ids_labels_test["coded_id"],on="coded_id",how="right")

res.to_csv("res.csv",index=None)
res

Score de précision pour Naive Bayes :  0.8623188405797102
Le rappel pour Naive Bayes :  0.68
[0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0]
    label  coded_id  ...  urls_used_on_average  utc_offset
0     NaN         5  ...                 1.000     10800.0
1     NaN        26  ...                 1.000     -4837.5
2     NaN        37  ...                 1.211     10800.0
3     NaN        40  ...                 1.000     10800.0
4     NaN        52  ...                 2.556     -4837.5
..    ...       ...  ...                   ...         ...
76    NaN       729  ...                 1.130    -14400.0
77    NaN       745  ...                 1.023    -21600.0
78    NaN       746  ...                 1.042    -21600.0
79    NaN       757  ...            

,coded_id,label
0,5,0
1,26,0
2,37,0
3,40,0
4,52,1
...,...,...
76,729,0
77,745,0
78,746,0
79,757,0
